In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

In [46]:
msg = pd.read_csv('datasets/messages.csv')
msg.head(4)

,CONVERSATION ID,CONVERSATION TITLE,FROM,SENDER PROFILE URL,TO,RECIPIENT PROFILE URLS,DATE,SUBJECT,CONTENT,FOLDER
0,2-MGIyMDY4NDMtYjU1YS00ODY3LTkyMjQtY2I5MTMxZWIw...,NaN,Gaston Ariel Garcia,https://www.linkedin.com/in/gaston-ariel-garci...,Juan Martin Diaz,https://www.linkedin.com/in/juan-martin-diaz,2023-08-22 21:33:15 UTC,NaN,Hola . gaston.garcia@medlog.com,INBOX
1,2-MGIyMDY4NDMtYjU1YS00ODY3LTkyMjQtY2I5MTMxZWIw...,NaN,Juan Martin Diaz,https://www.linkedin.com/in/juan-martin-diaz,Gaston Ariel Garcia,https://www.linkedin.com/in/gaston-ariel-garci...,2023-08-22 13:25:29 UTC,NaN,"Gaston, te parecería que te pase esta info a a...",INBOX
2,2-MGIyMDY4NDMtYjU1YS00ODY3LTkyMjQtY2I5MTMxZWIw...,NaN,Gaston Ariel Garcia,https://www.linkedin.com/in/gaston-ariel-garci...,Juan Martin Diaz,https://www.linkedin.com/in/juan-martin-diaz,2023-08-17 15:41:07 UTC,NaN,"Perfecto, lo tengo presente por cualquier nec...",INBOX
3,2-MGIyMDY4NDMtYjU1YS00ODY3LTkyMjQtY2I5MTMxZWIw...,NaN,Gaston Ariel Garcia,https://www.linkedin.com/in/gaston-ariel-garci...,Juan Martin Diaz,https://www.linkedin.com/in/juan-martin-diaz,2023-08-17 15:40:47 UTC,NaN,Hola Juan.,INBOX


In [47]:
msg.tail(29)

,CONVERSATION ID,CONVERSATION TITLE,FROM,SENDER PROFILE URL,TO,RECIPIENT PROFILE URLS,DATE,SUBJECT,CONTENT,FOLDER
2186,2-MDkxZDM4NjUtNzgyYS00NWZmLWIzOGUtM2UwMjRlZTNi...,NaN,Juan Martin Diaz,https://www.linkedin.com/in/juan-martin-diaz,Camila Fonseca,https://www.linkedin.com/in/camila-fonseca-811...,2023-07-19 17:56:46 UTC,NaN,Camila como estas? Un gusto conectar. Me post...,ARCHIVE
2187,2-MjE4NzU0ZWYtMGRjZS00YTliLWFiYWItNzdhNmNlNjFh...,Sponsored Conversation,LinkedIn Member,NaN,Juan Martin Diaz,https://www.linkedin.com/in/juan-martin-diaz,2023-07-03 09:39:50 UTC,NaN,"<p class=""spinmail-quill-editor__spin-break"">H...",ARCHIVE
2188,2-ZjViNDBmOWYtMmE0ZC00NWZmLTkwNzYtODIzNWVhZjRm...,Sponsored Conversation,LinkedIn Member,NaN,Juan Martin Diaz,https://www.linkedin.com/in/juan-martin-diaz,2023-05-19 03:28:29 UTC,NaN,"Hola, %FIRSTNAME%: Me llamo Eugene. Acabamos ...",ARCHIVE
2189,2-MWYxMTc3OWMtYTJlZS00MjM5LWI5NDAtNTg2ZWM3YzEz...,Sponsored Conversation,LinkedIn Member,NaN,Juan Martin Diaz,https://www.linkedin.com/in/juan-martin-diaz,2023-04-27 20:42:51 UTC,NaN,"<p class=""spinmail-quill-editor__spin-break"">H...",ARCHIVE
2190,2-YWQ4ZjAzMzAtM2JjMC00YWZkLWFkMzMtNDJiNWNhYjBm...,Sponsored Conversation,LinkedIn Member,NaN,Juan Martin Diaz,https://www.linkedin.com/in/juan-martin-diaz,2023-04-04 02:49:23 UTC,NaN,"Hola, %FIRSTNAME%: Soy André, gerente de mark...",ARCHIVE
2191,2-NGI3ODJlOWMtZWQ2Yi00NzM4LWE3ODItOTg2YWY1ODZi...,"Baldor SRL ""Laboratorio de Calibración""",Fernando Dolber,NaN,Juan Martin Diaz,https://www.linkedin.com/in/juan-martin-diaz,2023-03-21 01:52:53 UTC,NaN,"<p class=""spinmail-quill-editor__spin-break""><...",ARCHIVE
2192,2-NzZhMjI3ZDItYzJmZS00OThiLWI0NzUtZDQ3ZTFiN2I2...,Sponsored Conversation,LinkedIn Member,NaN,Juan Martin Diaz,https://www.linkedin.com/in/juan-martin-diaz,2023-02-16 14:08:52 UTC,NaN,"Hola, %FIRSTNAME%: Más candidatos adecuados t...",ARCHIVE
2193,2-NGNjNWVkOGItZThiZC00ZWI4LTljYmUtMmFmZWM2NzE3...,Sponsored Conversation,LinkedIn Member,NaN,Juan Martin Diaz,https://www.linkedin.com/in/juan-martin-diaz,2023-01-02 11:19:04 UTC,NaN,"Hola, %FIRSTNAME%: Más candidatos adecuados t...",ARCHIVE
2194,2-MTkzMzQ2YmMtNjQ4YS00ZWVlLWE0MmMtMDIxOTAyNjBm...,La campaña de Volkswagen que los condujo al éxito,Mercado Ads,NaN,Juan Martin Diaz,https://www.linkedin.com/in/juan-martin-diaz,2022-12-29 17:08:33 UTC,NaN,"<p class=""spinmail-quill-editor__spin-break"">H...",ARCHIVE
2195,2-ZjRhYmQ4OWQtNDI5Ni00NjM5LWJmZWMtZjVhNjM0NzVm...,Conocé la exitosa estrategia regional de L’Oréal,Mercado Ads,NaN,Juan Martin Diaz,https://www.linkedin.com/in/juan-martin-diaz,2022-12-08 15:03:26 UTC,NaN,"<p class=""spinmail-quill-editor__spin-break"">H...",ARCHIVE


Viendo algunos de los ultimos registros de nuestra base de datos, podemos concluir que allí se situan mensajes que consideraremos como Spam, ya que provienen de cuentas de Linkedin Premium, Linkedin Learning y otras cuentas, las que no aportan nada a nuestro analisis.

Borraremos estos registros, que son los ultimos 28, es decir, desde el index 2187 al 2214

In [48]:
registros_a_eliminar = [i for i in range(2187,2215)]
msg.drop(registros_a_eliminar, inplace=True)

Mirandolo mejor, me estoy dando cuenta que los datos estan ordenados por fecha de forma descendente

In [49]:
msg.shape
# es decir, tenemos un total de 2215 mensajes recibidos/enviados en nuestra casilla 

(2187, 10)

In [50]:
msg.columns

Index(['CONVERSATION ID', 'CONVERSATION TITLE', 'FROM', 'SENDER PROFILE URL',
       'TO', 'RECIPIENT PROFILE URLS', 'DATE', 'SUBJECT', 'CONTENT', 'FOLDER'],
      dtype='object')

In [51]:
msg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2187 entries, 0 to 2186
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   CONVERSATION ID         2187 non-null   object 
 1   CONVERSATION TITLE      2 non-null      object 
 2   FROM                    2180 non-null   object 
 3   SENDER PROFILE URL      2178 non-null   object 
 4   TO                      2179 non-null   object 
 5   RECIPIENT PROFILE URLS  2179 non-null   object 
 6   DATE                    2187 non-null   object 
 7   SUBJECT                 0 non-null      float64
 8   CONTENT                 2150 non-null   object 
 9   FOLDER                  2187 non-null   object 
dtypes: float64(1), object(9)
memory usage: 171.0+ KB


In [52]:
# removeremos las columnas que no nos sirven
msg.drop(columns=['CONVERSATION TITLE', 'SUBJECT', 'SENDER PROFILE URL', 'RECIPIENT PROFILE URLS'], inplace=True)
msg

,CONVERSATION ID,FROM,TO,DATE,CONTENT,FOLDER
0,2-MGIyMDY4NDMtYjU1YS00ODY3LTkyMjQtY2I5MTMxZWIw...,Gaston Ariel Garcia,Juan Martin Diaz,2023-08-22 21:33:15 UTC,Hola . gaston.garcia@medlog.com,INBOX
1,2-MGIyMDY4NDMtYjU1YS00ODY3LTkyMjQtY2I5MTMxZWIw...,Juan Martin Diaz,Gaston Ariel Garcia,2023-08-22 13:25:29 UTC,"Gaston, te parecería que te pase esta info a a...",INBOX
2,2-MGIyMDY4NDMtYjU1YS00ODY3LTkyMjQtY2I5MTMxZWIw...,Gaston Ariel Garcia,Juan Martin Diaz,2023-08-17 15:41:07 UTC,"Perfecto, lo tengo presente por cualquier nec...",INBOX
3,2-MGIyMDY4NDMtYjU1YS00ODY3LTkyMjQtY2I5MTMxZWIw...,Gaston Ariel Garcia,Juan Martin Diaz,2023-08-17 15:40:47 UTC,Hola Juan.,INBOX
4,2-MGIyMDY4NDMtYjU1YS00ODY3LTkyMjQtY2I5MTMxZWIw...,Juan Martin Diaz,Gaston Ariel Garcia,2023-08-17 12:44:50 UTC,"Buen día Gaston, como estas? Un gusto conectar...",INBOX
...,...,...,...,...,...,...
2182,2-NjRkYjhmNjUtNmRmNC00YThhLWJmNjQtNDQ0Y2FiZjVm...,Ruben Lopez,Juan Martin Diaz,2021-12-18 09:54:35 UTC,Gracias,INBOX
2183,2-NjRkYjhmNjUtNmRmNC00YThhLWJmNjQtNDQ0Y2FiZjVm...,Juan Martin Diaz,Ruben Lopez,2021-12-14 02:51:34 UTC,¡Felicidades por tu aniversario de trabajo!,INBOX
2184,2-MTI2ZDU5YmItNTE3NC00NjNjLThmNzktYmRiNDlkZDAx...,Juan Martin Diaz,Fernando Aguirre,2021-12-07 21:02:22 UTC,Gracias,INBOX
2185,2-MTI2ZDU5YmItNTE3NC00NjNjLThmNzktYmRiNDlkZDAx...,Fernando Aguirre,Juan Martin Diaz,2021-12-07 17:35:03 UTC,¡Felicidades por tu nuevo puesto!,INBOX


In [53]:
msg['FROM'].value_counts()


FROM
Juan Martin Diaz            1392
Nicolás Coria                 29
Maria Laura Gauna Fredes      22
Joaquin Gamez Maggiori        21
Juan cruz Vittori             16
                            ... 
Kevin Olguin                   1
Matías Garciandia              1
Carolina Funes                 1
Daiana Turchi                  1
Fernando Aguirre               1
Name: count, Length: 242, dtype: int64

In [54]:
msg['DATE'] = pd.to_datetime(msg['DATE']) # transformamos a tipo fecha
msg['DATE'] = msg['DATE'].dt.strftime(date_format='%Y-%m-%d %H:%M:%S') # quitamos el +00:00 de la zona horaria
msg['DATE']

0       2023-08-22 21:33:15
1       2023-08-22 13:25:29
2       2023-08-17 15:41:07
3       2023-08-17 15:40:47
4       2023-08-17 12:44:50
               ...         
2182    2021-12-18 09:54:35
2183    2021-12-14 02:51:34
2184    2021-12-07 21:02:22
2185    2021-12-07 17:35:03
2186    2023-07-19 17:56:46
Name: DATE, Length: 2187, dtype: object

La cuenta de Linkedin puede tener muchos mensajes que no son de utilidad para nuestro analisis, la fecha de analisis es algo que se debera tener en cuenta. 

Esto es así porque por ejemplo, en el año 2021 la empresa no siquiera estaba realizando esta campaña de marketing, por eso con el objetivo de mejorar la calidad de los datos vamos a fijar una fecha para analizar los datos de ahi en adelante
- Fecha de comienzo de campaña: 2022-04-01

In [63]:
msg = msg.loc[msg['DATE']>'2022-04-01'].reset_index(drop=True)


In [65]:
len(msg) # notar que pasamos de tener 2215 registros a tener 2099

2099

El siguiente paso seria crear una api con fastapi y render para mostrar en vivo un analisis de los datos.